# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2023-05-24	Predicting Glaucoma Progression to Surgery with Artificial Intelligence Survival Models	Ophthalmology Science	"In the present study, we developed survival-based AI models for predicting glaucoma patients' progression to surgery, comparing performance among regression-, tree-, and deep-learning-based approaches."	S. Tao, R. Ravindranath, and S. Y. Wang, "Predicting Glaucoma Progression to Surgery with Artificial Intelligence Survival Models",Ophthalmology Science, p. 100336, 2023.	glaucoma-progression-survival-models	https://www.ophthalmologyscience.org/article/S2666-9145(23)00068-4/fulltext
2023-04-01	Differences in Ideal Cardiovascular Health Between Sexual Minority and Heterosexual Adults	JAMA Cardiology	"Research on the cardiovascular health (CVH) of sexual minority adults has primarily examined differences in the prevalence of individual CVH metrics rather than comprehensive measures, which has limited development 

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2023-05-24,Predicting Glaucoma Progression to Surgery wit...,Ophthalmology Science,"In the present study, we developed survival-ba...","S. Tao, R. Ravindranath, and S. Y. Wang, ""Pred...",glaucoma-progression-survival-models,https://www.ophthalmologyscience.org/article/S...
1,2023-04-01,Differences in Ideal Cardiovascular Health Bet...,JAMA Cardiology,Research on the cardiovascular health (CVH) of...,"B. A. Caceres, Y. Sharma, R. Ravindranath et a...",cardiovascular-differences-sgm,https://jamanetwork.com/journals/jamacardiolog...
2,2023-11-16,Social Determinants of Health and Perceived Ba...,JAMA Ophthalmology,This study evaluated the associations between ...,"Ravindranath R, Bernstein IA, Fernandez KS, Lu...",diabetic-retinopathy-screening-analysis,https://jamanetwork.com/journals/jamaophthalmo...
3,2022-10-30,An Investigation of Differences in Ideal Cardi...,Circulation,"In this study, we analyzed cross-sectional dat...","Caceres BA, Sharma Y, Ravindranath R, et al. ""...",sgm-investigation-nhanes-abstract,https://www.ahajournals.org/doi/10.1161/circ.1...
4,2023-11-06,An Examination of Differences in Ideal Cardiov...,Circulation,We analyzed data from the All of Us Research P...,"D Doan, Ravindranath R, J Belloir, V Nguyen e...",cvd-differences-allofus,https://www.ahajournals.org/doi/abs/10.1161/ci...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2022-10-30-sgm-investigation-nhanes-abstract.md
2023-04-01-cardiovascular-differences-sgm.md
2023-05-24-glaucoma-progression-survival-models.md
2023-11-06-cvd-differences-allofus.md
2023-11-16-diabetic-retinopathy-screening-analysis.md


In [7]:
!cat ../_publications/2023-05-24-glaucoma-progression-survival-models.md

---
title: "Predicting Glaucoma Progression to Surgery with Artificial Intelligence Survival Models"
collection: publications
permalink: /publication/2023-05-24-glaucoma-progression-survival-models
excerpt: 'In the present study, we developed survival-based AI models for predicting glaucoma patients&apos; progression to surgery, comparing performance among regression-, tree-, and deep-learning-based approaches.'
date: 2023-05-24
venue: 'Ophthalmology Science'
paperurl: 'https://www.ophthalmologyscience.org/article/S2666-9145(23)00068-4/fulltext'
citation: 'S. Tao, R. Ravindranath, and S. Y. Wang, &quot;Predicting Glaucoma Progression to Surgery with Artificial Intelligence Survival Models&quot;,Ophthalmology Science, p. 100336, 2023.'
---
In the present study, we developed survival-based AI models for predicting glaucoma patients&apos; progression to surgery, comparing performance among regression-, tree-, and deep-learning-based approaches.

[Download paper here](https://www.ophthalmo